In [ ]:
from ccmm.utils.utils import load_model_from_artifact
import wandb
from nn_core.model_logging import NNLogger
from nn_core.serialization import NNCheckpointIO
import torch
import os
import sys
import pytorch_lightning as pl

In [ ]:
run = wandb.init(project="cycle-consistent-model-merging", entity="gladia", job_type="dev")

artifact_path = f"gladia/cycle-consistent-model-merging/VGG16_3:v1"

# {a: model_a, b: model_b, c: model_c, ..}
model = load_model_from_artifact(run, artifact_path)

In [ ]:
trainer = pl.Trainer(
    plugins=[NNCheckpointIO(jailing_dir="./tmp")],
)

temp_path = "temp_checkpoint.ckpt"

trainer.strategy.connect(model)
trainer.save_checkpoint(temp_path)

model_class = model.__class__.__module__ + "." + model.__class__.__qualname__

artifact_name = f""
model_artifact = wandb.Artifact(
    name="CIFAR100_VGG16_3",
    type="checkpoint",
    metadata={"model_identifier": "VGG16", "model_class": model_class},
)

model_artifact.add_file(temp_path + ".zip", name="trained.ckpt.zip")
run.log_artifact(model_artifact)

os.remove(temp_path + ".zip")